<a href="https://colab.research.google.com/github/VarunMani04/Datapipeline-Project/blob/main/DataPipelineProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Podcast Parameters & Logic

In [ ]:
#Podcast: Marketplace Tech
Link: "https://www.marketplace.org/feed/podcast/marketplace-tech"

from airflow.decorators import dag, task
import pendulum

import requests #xml podcast data processor
import xmltodict

@dag(
    start_date = pendulum.datetime(2023,9,1)
    dag_id = "podcast_summary"
    schedule_interval = "@daily" #collected daily
    catchup = False
)

Creating a Database

In [ ]:
from airflow.providers.sqlite.operators.sqlite import SqliteOperator

  def podcast_summary():
    create_database = SqliteOperator( #Pass in SQL Code
        task_id='create_table_sqlite',
        sql=r"""
        CREATE TABLE IF NOT EXISTS episodes (
            link TEXT PRIMARY KEY,
            title TEXT,
            filename TEXT,
            published TEXT,
            description TEXT,
            transcript TEXT
        );
        """,
        sqlite_conn_id="podcasts" #tells airflow which connection to use
    )
    @task()


Downloading Data

In [ ]:
def podcast_logic(): #base logic for pipeline
  @task()

  def get_podcasts(Link):
    data = requests.get()
    feed = xmltodict.parse(data.text)
    names = feed["rss"]["channel"]["item"] #parses through dictionary to names
    return ("Episode names: ", names )


  pipeline = podcast_logic() #pipeline object initalized
  episodes = get_podcasts():
  create_databse.set_downstream(episodes)

Load Episodes into Database Avoiding Duplicates

In [ ]:
from airflow.providers.sqlite.hooks.sqlite import SqliteHook

def load_episodes(episodes):
  hook = SqliteHook(splite_conn_id="podcasts")
  stored = hook.get_pandas_df("SELECT * from episodes;") #figure out what episodes have been stored
  new_episodes = [] #if not add to database

  for episode in episodes: #process through each episode
    if episode["link"] not in stored["link"].values:
      filename = f"{episode["link"].split("/"[-1])}.mp3"
      new_episodes.append([episode["link"], episode["title"], episode["pubDate"], episode["description"], filename])
  hook.insert_rows(table='episodes', rows=new_episodes, target_fields=["link", "title", "published", "description", "filename"]) #Each new episode will have these parameters

load_episodes(episodes)

Download Episodes

In [ ]:
import os

@task() #Airflow feature that automates function tasks
def download_episodes(episodes):
  for episode in episodes:
    filename = f"{episode["link"].split("/")[-1]}.mp3" #fstring to convert to mp3
    audio = os.path.join()

finalSum = podcast_summary()